In [258]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [326]:
match = pd.read_csv("results.csv")

In [327]:
#put date in correct format and set it as the index
date = pd.to_datetime(match.date.values)
match['date'] =date
match.set_index('date', inplace=True)

In [328]:
#get the coloumn of results (wins, ties and losses)

win= np.where(match.home_score > match.away_score, 'win', None)

tie=np.where(match.home_score == match.away_score, 'tie', None)

loss= np.where(match.home_score < match.away_score, 'loss', None)

In [329]:
results = pd.DataFrame([win, tie, loss]).T

x = [value[value != None]  for value in results.values]
 
match['result'] = x
match['result'] = np.squeeze(match.result.tolist())

In [330]:
#get the number of goals
match['goals']= match.home_score + match.away_score

In [331]:
#home
home_teams = match.groupby(['home_team','result']).count()['city'].sort_values(ascending=False).reset_index().rename(columns={'city': 'count'})

In [332]:
home_matches=[]

for team in home_teams.home_team:
    tot_matches= home_teams[home_teams.home_team== team]['count'].sum()
    home_matches.append(tot_matches)
   
home_teams['home_matches']=home_matches
home_teams['pct_home_victory']= home_teams['count']/ home_teams['home_matches']

In [333]:
#away

away_teams=match.groupby(['away_team','result']).count()['city'].sort_values(ascending=False).reset_index().rename(columns={'city': 'count'})
away_teams.replace({'loss': 'win', 'win':'loss'}, inplace=True)  

In [334]:
#loss means victory for the away team

away_tot_matches=[]

for team in away_teams.away_team:
    tot_matches= away_teams[away_teams.away_team == team]['count'].sum()
    away_tot_matches.append(tot_matches)

away_teams['away_matches']= away_tot_matches
away_teams['pct_victory_away'] = away_teams['count']/away_teams['away_matches']


In [335]:
#adjusting terminology and index

home_teams.rename(columns={'result': 'home_results', 'count': 'home_count'}, inplace=True)
home_teams.set_index('home_team', inplace=True)

away_teams.rename(columns={'result': 'away_results', 'count': 'away_count'}, inplace=True)
away_teams.set_index('away_team', inplace=True)

In [336]:
#defining winners and loosers

home_winners= home_teams[home_teams.home_results=='win']
away_winners= away_teams[away_teams.away_results=='win']

home_losers= home_teams[home_teams.home_results=='loss']
away_losers= away_teams[away_teams.away_results=='loss']

In [337]:
#merging datasets

winners=pd.merge(home_winners, away_winners, left_index=True, right_index=True, how='inner')
losers=pd.merge(home_losers, away_losers, left_index=True, right_index=True, how='inner')
losers.rename(columns={'pct_home_victory': 'pct_home_defeats', 'pct_victory_away': 'pct_away_defeats'}, inplace=True)

winners['tot_count']= winners.home_count + winners.away_count
winners['tot_matches']= winners.home_matches + winners.away_matches
winners['tot_pct_victory']= winners.tot_count/winners.tot_matches

#getting only clubs who have played at least 100 matches
winners= winners[winners.tot_matches >= 100] 
winners_pct= winners[['pct_home_victory', 'pct_victory_away', 'tot_pct_victory']]

losers['tot_count']= losers.home_count + losers.away_count
losers['tot_matches']= losers.home_matches + losers.away_matches
losers['tot_pct_defeats']= losers.tot_count/losers.tot_matches

#getting only clubs who have played at least 100 matches
losers= losers[losers.tot_matches >= 100] 
losers_pct= losers[['pct_home_defeats', 'pct_away_defeats', 'tot_pct_defeats']]

In [338]:
#total percentage
winners_pct.sort_values(by='tot_pct_victory', ascending=False)
winners_pct=np.round(winners_pct*100, 2)
winners_pct['tot_count']= winners.tot_count
winners_pct['tot_matches']= winners.tot_matches


losers_pct=np.round(losers_pct*100, 2)
losers_pct['tot_count']= losers.tot_count
losers_pct['tot_matches']= losers.tot_matches

winners_pct.sort_values(by='tot_pct_victory', ascending=False)

,pct_home_victory,pct_victory_away,tot_pct_victory,tot_count,tot_matches
Brazil,71.13,53.51,63.71,625,981
Spain,68.25,48.06,58.50,406,694
Germany,62.65,53.72,58.48,555,949
England,61.85,51.66,56.69,572,1009
Iran,63.21,43.38,54.51,272,499
...,...,...,...,...,...
Malta,15.98,6.74,11.84,47,397
Seychelles,21.57,3.23,11.50,13,113
Somalia,13.64,6.35,9.35,10,107
Luxembourg,11.26,5.00,8.46,34,402


In [342]:
#discover goal and history

home_matches = match.groupby('home_team').sum().rename(columns={'home_score': 'goals_scored_home', 'away_score' : 'goals_taken_home'})

away_matches = match.groupby('away_team').sum().rename(columns={'home_score': 'goals_taken_away', 'away_score' : 'goals_scored_away'})


In [343]:
#merging 

score= pd.merge(home_matches, away_matches, how='inner', right_index= True, left_index=True)


In [344]:
#creating calculated fields
score['tot_goals_scored']= score['goals_scored_home'] + score['goals_scored_away']
score['tot_goals_taken']= score['goals_taken_home'] + score['goals_taken_away']
score['diff_goals_tot'] = score['tot_goals_scored'] - score['tot_goals_taken']
score.drop(columns=['neutral_x', 'goals_x', 'neutral_y', 'goals_y'], inplace=True)


In [345]:
#get the match count (only teams with 100 match played are taken)
score=pd.merge(score, winners[['home_matches', 'away_matches', 'tot_matches']], how='inner', right_index=True, left_index=True)


In [346]:
#compute ratios
score['home_goals_pct'] = score['goals_scored_home']/ score['home_matches']
score['away_goals_pct'] = score['goals_scored_away']/ score['away_matches']
score['home_goals_taken_pct'] = score['goals_taken_home']/ score['home_matches']
score['away_goals_taken_pct'] = score['goals_taken_away']/ score['away_matches']
score['tot_goals_scored_pct'] = score['tot_goals_scored']/ score['tot_matches']
score['tot_goals_taken_pct'] = score['tot_goals_taken']/ score['tot_matches']
score_pct= score[['home_goals_pct', 'away_goals_pct', 'home_goals_taken_pct', 'away_goals_taken_pct',
                 'tot_goals_scored_pct', 'tot_goals_taken_pct', 'diff_goals_tot']]

score_pct.sort_values(by='diff_goals_tot', ascending=False)

,home_goals_pct,away_goals_pct,home_goals_taken_pct,away_goals_taken_pct,tot_goals_scored_pct,tot_goals_taken_pct,diff_goals_tot
Brazil,2.470070,1.806295,0.829225,1.021792,2.190622,0.910296,1256
England,2.307229,2.072407,0.897590,1.070450,2.188305,0.985134,1214
Germany,2.462451,1.981941,1.031621,1.291196,2.238145,1.152792,1030
Argentina,2.200730,1.444444,0.872263,1.268519,1.867347,1.046939,804
Spain,2.350975,1.659701,0.743733,1.071642,2.017291,0.902017,774
...,...,...,...,...,...,...,...
Cyprus,1.108374,0.597484,1.704433,2.849057,0.883978,2.207182,-479
Northern Ireland,1.282051,0.825301,1.689103,2.081325,1.046584,1.891304,-544
Malta,0.780822,0.393258,1.799087,2.943820,0.607053,2.312343,-677
Finland,1.460606,0.946860,1.924242,2.241546,1.174731,2.100806,-689


In [347]:
#join winners with score

total_ratio = pd.merge(score_pct, winners_pct, how = 'inner', right_index=True, left_index=True)
total_ratio = pd.merge(total_ratio, losers_pct, how = 'inner', right_index=True, left_index=True)

score_pct.sort_values(by ='diff_goals_tot', ascending = False)

correlation = total_ratio[['tot_goals_scored_pct', 'tot_goals_taken_pct', 
                          'tot_pct_victory','tot_pct_defeats']].corr()

In [348]:
total_ratio

,home_goals_pct,away_goals_pct,home_goals_taken_pct,away_goals_taken_pct,tot_goals_scored_pct,tot_goals_taken_pct,diff_goals_tot,pct_home_victory,pct_victory_away,tot_pct_victory,tot_count_x,tot_matches_x,pct_home_defeats,pct_away_defeats,tot_pct_defeats,tot_count_y,tot_matches_y
Afghanistan,1.275000,0.887324,1.525000,2.281690,1.027027,2.009009,-109,42.50,18.31,27.03,30,111,37.50,59.15,51.35,57,111
Albania,1.114286,0.700637,1.154286,2.050955,0.918675,1.578313,-219,35.43,14.01,25.30,84,332,40.57,68.79,53.92,179,332
Algeria,1.784247,0.873239,0.856164,1.272300,1.400000,1.031683,186,55.14,24.88,42.38,214,505,20.21,43.66,30.10,152,505
Angola,1.432749,0.856354,0.865497,1.243094,1.136364,1.059659,27,47.95,21.55,34.38,121,352,19.30,41.44,30.68,108,352
Antigua and Barbuda,1.541284,1.218391,1.330275,2.344828,1.397959,1.780612,-75,43.12,17.24,31.63,62,196,38.53,62.07,48.98,96,196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yemen,1.247525,0.712000,1.475248,2.912000,0.951327,2.269912,-298,31.68,10.40,19.91,45,226,43.56,75.20,61.06,138,226
Yugoslavia,2.310526,1.727586,1.242105,1.727586,1.958333,1.535417,203,56.84,39.66,46.46,223,480,23.68,39.31,33.12,159,480
Zambia,1.876667,1.334928,0.833333,1.165072,1.561281,1.026462,384,55.00,39.00,45.68,328,718,16.00,36.36,27.86,200,718
Zanzibar,1.086207,0.843537,1.724138,2.272109,0.912195,2.117073,-247,29.31,16.33,20.00,41,205,53.45,62.59,60.00,123,205


In [349]:
#machine learining to predict victory/loss rate

total_ratio.corr()['tot_pct_victory'] #check correlation
train = np.array(total_ratio.iloc[:, :7])
test = np.array(total_ratio['tot_pct_victory'])

In [353]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, test, random_state=42)

In [354]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train,y_train)

0.935416311154903

In [357]:
lr.score(X_test, y_test)

0.9348934437746117

In [355]:
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor(n_estimators=200, max_features=2, max_depth=20, bootstrap=True, random_state=1)
rf.fit(X_train, y_train)
rf.score(X_train,y_train)

0.9897881762134753

In [356]:
rf.score(X_test,y_test)

0.9583448182900143

In [358]:
#goals through the years

year_goals= match.groupby(by=[match.index.year]).sum()['goals']
year_match= match.groupby(by=[match.index.year]).count()['neutral']
year = pd.DataFrame([year_goals, year_match]).T.rename(columns={'neutral': 'matches'})

#normalize
year['goals_match'] = year.goals/year.matches 
year = np.round(year, 2)

In [362]:
year 

,goals,matches,goals_match
date,,,
1872,0,1,0.00
1873,6,1,6.00
1874,3,1,3.00
1875,4,1,4.00
1876,7,2,3.50
...,...,...,...
2016,2531,926,2.73
2017,2583,958,2.70
2018,2359,905,2.61
